In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
import time

try:
    driver = webdriver.Chrome()
except Exception as e:
    print(str(e))
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
    except Exception as e:
        print(str(e))
# Set timeouts
page_load_timeout = 25
element_presence_timeout = 20
driver.set_page_load_timeout(page_load_timeout)

driver.get("https://web.whatsapp.com/")

# Click on first Chat

In [87]:
# Click on the first chat
# element = driver.find_element(By.CSS_SELECTOR, "#pane-side > div:nth-child(2) > div > div > div:nth-child(1) > div > div > div > div._ak8l._ap1_")
# element.click()

element = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[3]/div/div[3]/div/div[3]/div[1]/div/div/div[1]/div/div/div/div[2]")
element.click()

# Click on Group Info

In [88]:
time.sleep(5)

# Click on the header element
element = driver.find_element(
    By.CSS_SELECTOR,
    "#main > header > div.x78zum5.xdt5ytf.x1iyjqo2.xl56j7k.xeuugli.xtnn1bt.x9v5kkp.xmw7ebm.xrdum7p > div.x78zum5.x1cy8zhl.xisnujt.x1nxh6w3.xcgms0a.xhslqc4"
)
element.click()

# Click on Group Members

In [89]:
time.sleep(2)

# Click on the button
button = driver.find_element(
    By.CSS_SELECTOR,
    "#app > div._aiwn._aiwm.app-wrapper-web.font-fix.os-win.x1h89ln0.xoeyzqq > div > div.x78zum5.xdt5ytf.x5yr21d > div > div._aig-._as6h.x9f619.x1n2onr6.x5yr21d.x6ikm8r.x10wlt62.x17dzmu4.x1i1dayz.x2ipvbc.xjdofhw.xpilrb4.x1t7ytsu.x1vb5itz.x1c4vz4f.x2lah0s.x1oy9qf3.xwfak60.x5hsz1j.x17dq4o0.x10e4vud > span > div > span > div > div > div > section > div.x13mwh8y.x1q3qbx4.x1wg5k15.x3psx0u.xat24cr.x1280gxy.x1h678fw.xcldk2z.xv6tirj.x1phvje8 > div > div.x1evy7pa.x1kgmq87.x2b8uid > span > div > button"
)
button.click()

# Click on Members (Open Members info Pop-Up )

In [90]:
time.sleep(2)

# --- Locate the element and click it ---
target = driver.find_element(
    By.XPATH,
    '//*[@id="app"]/div[1]/div/div[3]/div/div[5]/span/div/span/div/div/div/section/div[10]/div[1]/div/div[1]/span'
)
target.click()


# Scrolling Down & Extract Mobile Numbers

In [91]:
scroll_div = driver.find_element(
    By.XPATH,
    "/html/body/div[1]/div/div/div[1]/div/span[2]/div/span/div/div/div/div/div/div/div[2]"
)

pattern = r"\+91[\s\-]?\d{5}[\s\-]?\d{5}"

collected_numbers = set()
start_time = time.time()
TIMEOUT = 400          # max scroll time (seconds)
SCROLL_PAUSE = 1.1    # wait after each scroll
NO_CHANGE_LIMIT = 3   # stop if no change after 3 scrolls

no_change_count = 0
last_scroll = -1

while True:
    # Scrape visible text
    visible_text = scroll_div.text
    new_numbers = re.findall(pattern, visible_text)
    for num in new_numbers:
        collected_numbers.add(num.strip())

    # Scroll info
    current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_div)
    scroll_height = driver.execute_script("return arguments[0].scrollHeight;", scroll_div)
    client_height = driver.execute_script("return arguments[0].clientHeight;", scroll_div)

    # Detect if we’ve stopped moving (avoid infinite loop)
    if abs(current_scroll - last_scroll) < 5:  # no significant scroll movement
        no_change_count += 1
    else:
        no_change_count = 0

    last_scroll = current_scroll

    # Stop conditions
    if current_scroll + client_height >= scroll_height - 5:
        print("✅ Reached bottom of popup.")
        break
    if (time.time() - start_time) > TIMEOUT:
        print("⏰ Timeout reached.")
        break
    if no_change_count >= NO_CHANGE_LIMIT:
        print("🛑 No more new content loading, stopping scroll.")
        break

    # Scroll step
    driver.execute_script("arguments[0].scrollTop += arguments[0].clientHeight / 1.2;", scroll_div)
    time.sleep(SCROLL_PAUSE)

print("Total Number Extracted:", len(collected_numbers))

✅ Reached bottom of popup.
Total Number Extracted: 848


In [92]:
import pandas as pd
df = pd.DataFrame(sorted(collected_numbers), columns=["mobile_number"])
df['mobile_number'] = df['mobile_number'].str.replace(" ", "")
df

,mobile_number
0,+916001469752
1,+916005769698
2,+916232590755
3,+916238169121
4,+916267849298
...,...
843,+919995087633
844,+919995131864
845,+919995521262
846,+919995704299


In [93]:
df.to_excel("Star Diecast 2.0_numbers.xlsx", index=False)